
grafana llm service?


In [ ]:
const { Remarkable } = require('remarkable')
const md = new Remarkable({ html: true, xhtmlOut: true, breaks: true })

const express = require('express')
const cors = require('cors')
const crypto = require('crypto');

function generateHash(data, algorithm = 'sha256', encoding = 'hex') {
  const hash = crypto.createHash(algorithm);
  hash.update(data);
  return hash.digest(encoding);
}

let TIMEOUT = 15 * 1000
let initialChatHistory
let session
let hashes = [

]
async function handleRequest(req, res) {
  if (!session) {
    const { createSession } = await importer.import('llama vision')
    session = await createSession(process.env.DEFAULT_MODEL || 'gemma-3-27b-it-Q6_K.gguf', req.body.messages[0].content)
    initialChatHistory = session.getChatHistory();
  }

  let start = Date.now()

  if (req.body.messages && req.body.messages.length) {
    req.body.message = req.body.messages[req.body.messages.length - 1].content
  }

  res.set('Connection', 'keep-alive');
  res.set('Keep-Alive', 'timeout=' + (TIMEOUT / 1000) + ', max=' + (TIMEOUT / 1000))

  let hash = generateHash(req.body.content || req.body.message)
  if (typeof hashes[hash] != 'undefined') {
    await new Promise(resolve => setTimeout(resolve, TIMEOUT - 5)) // make client wait for requesting
    return res.json(hashes[hash])
  }
  console.log('User: ' + (req.body.content || req.body.message))
  let result = await session.prompt(req.body.content || req.body.message, {
    //temperature: temperature,
    onTextChunk: function (text) {
      process.stdout.write(text)
      if (typeof hashes[hash] == 'undefined' && start + TIMEOUT - 5 < Date.now()) {
        hashes[hash] = { "id": "resp_" + hash, "status": "pending" }
        res.json(hashes[hash])
      }
    }
  })

  console.log('AI: ' + result)
  session.setChatHistory(initialChatHistory)

  let code = result.matchAll(/```(json|bash|javascript|code|markdown)*\n[\s\S]*?\n*```/gi)
  if (code) {
    result = result.replaceAll(/^```(json|bash|javascript|code|markdown)\n*|\n*```\s*$/gi, '')
  }

  if (typeof hashes[hash] != 'undefined') {
    hashes[hash] = { content: result, object: result }
    return
  }

  if (req.headers.accept == 'application/json') {
    return res.json({ content: result, object: result });
  }
  const mdHtml = md.render(result)
  return res.send(mdHtml)
}

async function startServer() {
  var app = express()
  app.use(cors({
    credentials: true,
    origin: '*'
  }))
  app.use(express.json({ limit: '20mb' }))
  app.post('/', handleRequest)
  app.post('/v1/chat/completions', handleRequest)

  let port = process.env.CHAT_PORT || 9393
  let server = app.listen(port, () => console.log('Listening on http://localhost:' + port))
  server.keepAliveTimeout = TIMEOUT;
  server.headersTimeout = TIMEOUT;
}

module.exports = startServer


TODO: make a metrics for prometheus and logging for loki output

spread the console format like [service][debug] message throughout the notebooks

log by replacing console.log in services by importing constants from logging service

